# Imports

In [ ]:
import re
import pandas as pd
import numpy as np


In [ ]:
df_posts = pd.read_csv("df_posts_cleaned.csv")


In [ ]:
df_labeled_comments = pd.read_csv('df_labeled_comments.csv')

In [ ]:
label_counts = df_labeled_comments['label'].value_counts()
label_counts

label
NTA    2666030
YTA    1058187
ESH     160924
NAH     103490
Name: count, dtype: int64

# Combining comments with posts

In [ ]:
df_comment = df_labeled_comments

In [ ]:
df_posts = df_posts.drop(columns = ['permalink'])

In [ ]:
df_comment

,Unnamed: 0,submission_id,label
0,1,yiplwk,YTA
1,2,yiplwk,YTA
2,3,yiplwk,YTA
3,4,yiplwk,YTA
4,5,yiplwk,YTA
...,...,...,...
3988626,8981807,17uu03h,NTA
3988627,8981808,17uu03h,NTA
3988628,8981865,17uu03h,NAH
3988629,8981871,17uu03h,YTA


In [ ]:
#lables
df_labeles = df_comment.groupby('submission_id')['label'].apply(list).reset_index()

In [ ]:
from collections import Counter

def choose_label(labels, threshold = 25):
    counts = Counter(labels)

    if counts['NAH'] >= threshold:
        return 'NAH'
    elif counts['ESH'] >= threshold:
        return 'ESH'
    else:
        return counts.most_common(1)[0][0]  # most common label



In [ ]:
df_labeles['ChosenLabel'] = df_labeles['label'].apply(choose_label)

In [ ]:
df_labeles = df_labeles.drop(columns = 'label')
df_labeles = df_labeles.rename(columns={'ChosenLabel': 'Label'})

# Assigning lables to posts

In [ ]:
df_labeles.shape

(30054, 2)

In [ ]:
df_posts.shape

(30054, 5)

In [ ]:
labeled_post_ids = df_labeles['submission_id'].unique()

In [ ]:
# dropping posts that wouln't get lables assigned
df_posts = df_posts[df_posts['submission_id'].isin(labeled_post_ids)]


In [ ]:
df_posts.shape

(30054, 5)

In [ ]:
labeled_df = pd.merge(df_posts, df_labeles, on='submission_id', how='inner')

In [ ]:
labeled_df.to_csv('labeled_df')


In [ ]:
labeled_df['Label'].value_counts()

Label
NTA    23153
YTA     4499
ESH     1661
NAH      741
Name: count, dtype: int64

# Creating Final Subset:
This subset will hold only 500 records of minority classes only (YTA, ESH, NAH)




In [ ]:
minorities = labeled_df[labeled_df['Label'] != 'NTA']
minorities_subset = minorities[:500]

In [ ]:
minorities_subset.to_csv('AITA_minorities_subset.csv')